# Wöhler analyzing tool
Developed by Mustapha Kassem in scope of a master thesis at TU München

## Pylife Woehler-curve evaluation script

### Initialization

In [ ]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy import stats, optimize
import mystic as my
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from os import path
import io
import sys, os
import json

sys.path.insert(0, os.path.abspath('..'))

from pylife.materialdata.woehler.analyzer import *
from pylife.materialdata.woehler.diagram import *
from pylife.materialdata.woehler.widgets import *

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import warnings
import pdb

### Data import
#### Data is made up of two columns:
 * The first column is made up of the load values
 * The scond column is made up of the load-cycle values
 

In [ ]:
file_name = 'woehler-test-data.csv'

##### Transforming data in excel to python arrays

In [ ]:
data = pd.read_csv(file_name, sep='\t')
data.columns=['loads', 'cycles']

##### For automization purposes
data = pd.read_excel('../data/Test_dat.xlsx')
data.columns=['loads', 'cycles']

In [ ]:
w = wwidget.data_head_tail()
print('Choose whether to visualize the head or tail of the data found in excel:')
w

##### Visualization of the data:
* It is possible to go back and pick the other choice 
* Rerun this cell to see it

In [ ]:
if w.value == 'Head of the data':
    display(data.head())
else: 
    display(data.describe())

#### 2. Enter the load cycle limit (The load-cycle value that seperates Fractures from Run-outs):
##### Note: in case the load cycle limit is the highest number found in the column set it to *max(data[:,1])*

In [ ]:
ld_cyc_lim = data['cycles'].max()
#ld_cyc_lim = 10e6

## Parameters

#### 3. Fill in the fixed parameter values. 
#### The empty parameters are the ones to be estimated.
##### Note: don't rerun the cell after typing the values

In [ ]:
tab = wwidget.WL_param()
display(tab)

In [ ]:
fixed_param, estim_param =  wwidget.WL_param_display(tab)

#### 4. Variable definition of class WoehlerCurve: (Variable name = WC_data)

In [ ]:
warnings.filterwarnings('ignore')
#pdb.set_trace()
WC_data = WoehlerCurve(data, ld_cyc_lim, fixed_param, estim_param)
WC_data.calc_woehler_curve_parameters()

#### 5. Paramter results of the chosen evaluation method
##### Colored parameters: estimated
##### Black/white paramters: fixed

In [ ]:
estim_param.update(WC_data.Mali_5p_result)
all(map(estim_param.pop, fixed_param))

wwidget.results_mali_probit(WC_data, fixed_param)

| Variables              || Explanation                                          |
| :-------------         ||:-------------                                        |
| `fixed_param`          || Fixed parameters (by user)                           |
| `estim_param`          || Estimated parameters (exluding the fixed parameters) |
|`WC_data.Mali_5p_result`|| Estimated & fixed parameters                         |
|`WC_data.Mali_2p_result`|| MLE likelihood functions for the infinite-life zone  |
|`WC_data.k`             || Basquin slope using linear regression                |
|`WC_data.TN`            || Scatter in load-cycle direction using pearl-chain    |
|`WC_data.TS`            || Scatter in load direction using Koeder formula       |
|`WC_data.Probit_result` || Probit parameter results                             |


## Visualization of Results
#### 5. Enter necessary description to personalize the diagram:
 * For a default diagram description set the value default = 1
 * Enter the units for the axes as a string variable [amp, ld_typ, unit]

##### x- and y-axis are dynamically scaled to fit the data in a diagram. If the User wishes to change them:
 * Enter the limits for the x- and y-axis [xlim_WL and ylim_WL]
 <br>
(example: xlim_WL = (4E5, 8E10) )

In [ ]:
default = 1

if default == 0:
    amp = 'Amplitude'
    ld_typ = 'Stress'
    unit = u'$N/mm^2$'
    xlim_WL = (round(min(WC_data.data.cycles)*0.4,-1), round(max(WC_data.data.cycles)*2,-1))
    ylim_WL = (round(min(WC_data.data.loads)*0.8,-1), round(max(WC_data.data.loads)*1.2,-1))
else:
    amp, ld_typ, unit, xlim_WL, ylim_WL = 0,0,0,0,0

#### 6. Choose the plot type to be visualized in the following cell

In [ ]:
w4 = wwidget.results_visual()
print('Choose a plot type and execute the following cell to visualize the results:')
w4

##### Note: It is possible to go back and pick another choice. Rerun the following cell to see it.

In [ ]:
if len(WC_data.ld_lvls_inf[0])<2 and w4.value == "Probability plot of the infinite zone":
    print("Warning:\n")
    print("Not enough load levels in the infinite-life zone for plotting the probability net")
else:
    PlotWC = PlotWoehlerCurve(WC_data, w4.value, 'Mali', amp, ld_typ, unit, xlim_WL, ylim_WL, default)

### Final Woehler-curve plot
#### 7. Plot of the complete Woehler curve.
#### Choose the value of $k_2$ to plot the figure.

In [ ]:
k_1 = WC_data.Mali_5p_result['k_1']
w5 = wwidget.inf_plot(WC_data, k_1)
print('Choose a value for k_2:')
w5

### 8. Choose the parameters to plot
- Probit: $k_{regression}$, $\frac{1}{T_N}$, $ND_{50}$, $SD_{50}$(Probit),$\frac{1}{T_S}$(Probit)
- Mali: Maximum likelihood parameters (including the parameters fixed by the user)

In [ ]:
w2 = wwidget.method_mali_probit()
w2

In [ ]:
if len(WC_data.ld_lvls_inf[0])<2 and w2.value == "Probit":
    print("Warning:\n")
    print("Not enough load levels in the infinite zone for the probit method, please choose the Mali")
    print("\nparameters for visualization")
else:
    SD50, ND50, TS = PlotWC.runout_zone_method(WC_data, method=w2.value, slope_chosen=w5.value)
    _ = PlotWC.final_curve_plot(WC_data, SD50, ND50, TS, w5.value, w2.value, amp, ld_typ, unit, xlim_WL, ylim_WL, default)

## BIC
#### Bayesian Information Criterion: is a criterion for model selection among a finite set of models;
#### the model with the lowest BIC is preferred.

In [ ]:
sum_lolli = WC_data.mali_sum_lolli(WC_data.Mali_5p_result['SD_50'], WC_data.Mali_5p_result['1/TS'], 
                                        WC_data.Mali_5p_result['k_1'], WC_data.Mali_5p_result['ND_50'], 
                                        WC_data.Mali_5p_result['1/TN'], WC_data.fractures,
                                        WC_data.zone_inf, WC_data.load_cycle_limit)

n_data_pt = data.shape[0]
bic = WC_data.bayes_inf_crit(sum_lolli, WC_data.p_opt, n_data_pt)
print('\nBIC =',bic)

## Export variable data

In [ ]:
with open('param_dict.json', 'w') as fp:
    json.dump({'Mali_%dp_result_estimated'%len(WC_data.p_opt):estim_param, 
               'Mali_%dp_result_fixed'%len(WC_data.p_opt):fixed_param,
               'Mali_2p_result':WC_data.Mali_2p_result,
               'Probit_result':WC_data.Probit_result, 'k_reg': WC_data.k, '1/TN prl-chn':WC_data.TN,
               'BIC':bic}, fp)

## Reload variable data

In [ ]:
with open('param_dict.json', 'r') as fp:
    dict_import = json.load(fp)
dict_import